In [1]:
## Standard Imports ##
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')

import numpy as np
import scipy as sp

import time as timeit
import datetime
import sys,os

## Particle Filter Imports ##
import particles
from particles import state_space_models as ssm
from particles import smc_samplers as ssp
from particles import distributions as dists
import warnings; warnings.simplefilter('ignore')  # hide warnings 

## Particle Filter Example
In this file, I develop an sequential monte carlo squared (SMC2) solution to the identification of a single-degree-of-freedom oscillator (SDOF), in which the nonlinear switch state is turned ON, essentially developing a Bouc-Wen SDOF system. The nonlinear stiffness contribution is turned on for this part of the example, as part of the switch state for this system. 

SMC2, which falls into the particle filter class of Bayesian filtering approximations, incorporates a small set of hyperparameters used to adjust the sampling properties of the embedded Monte Carlo samplers used in this technique. For this example, these parameters were primarily selected according to the suggestions by Chopin and Papaspiliopoulos in their book "An Introduction to Sequential Monte Carlo"<sup>1</sup>. Specific values associated with MC sample points were selected to ensure sufficient convergence on the particular case study. These variables include: 
 
1. $N_{\theta}$, stated as ```init_Nx``` in the code. This variable represents the number of particles used in the SMC layer on the parameters. 
2. $AR$, states as ```ar_to_increase_Nx``` in the code. This variable represents the acceptance rate of the PMMH kernals used in the SMC layer of the filter. It is typically set to a value around 0.1, such that when the acceptance rate of the PMMH kernels falls below 10%, the number of particles on the parameters, $N_{\theta}$, is doubled. 
3. $N_x$, stated as ```N``` in the code. This variable represents the number of particle filter samples on the states, where a particle filter is attached to each of the $N_{\theta}$ particles approximating the parameters in order to form an unbiased estimate of the intractable marginal likelihood, $p(y_k|\theta)$.
4. The resampling scheme, stated as ```resampling``` in the code. As noted in <sup>1</sup>, several valid options are available for the resampling scheme. In this case, we have chosen to use a 'systematic' resampling scheme.<sup>2</sup>
5. $ESS$, stated as ```ESSrmin``` in the code. This variable represents the effective sample size threshold. If the effective sampling size falls below this threshold, the $N_x$ particles approximating the states are resampled and then moved according to a particle Markov chain Monte Carlo (PMCMC) kernel that leaves the current target distribution invariant. This value is usually set to $N/2$. 

This example runs 50 inference trials using varied prior information on the parameters, simulating different assertions an experimentalist might make in a practical identification scenario. Outputs from this model include:
1. The mean and standard deviation of the state and log(parameters) over the inference period. 
2. The mean of the state and parameters over the inference period. 
3. The mode of the state and parameters over the inference period. 
4. The computational model response built from the inferred parameters with respect to the input signal used for inference.
5. The runtime for each inference trial. 

In running this example, I found that the majority of the candidate priors developed convergent posterior models given ```init_Nx = 10``` and ```N = 50``` particles at the initialization of the filter.

For some cases, including sample priors $[8,10,25,31,32,39,42,47]$, these initialization particles were not sufficient, and the number of samples needed to be increased at significantly higher computational cost. As such, this candidate prior was evaluated with ```init_Nx = 15``` and ```N = 75```, which was found to generate convergent behavior in the posterior model. 

Additional hyperparameters in an experimental scenario include the process noise covariance $Q$ and the process noise covariance $R$. However, these terms are assumed known for this problem. 

The SMC$^2$ implementation expressed herein is drawn from the python library particles<sup>3</sup>. Some small modifications were made to adapt the library to this problem, and are shown in detail in the code below. 

__Developed by__: Alana Lund (Purdue University) \
__Last Updated__: 22 Sept. 2021 \
__License__: AGPL-3.0

### References
<sup>1</sup> N. Chopin and O. Papaspiliopoulos. An Introduction to Sequential Monte Carlo. _Springer Series in Statistics_ (2020). 

<sup>2</sup> J. Hol, T. Schon, and F. Gustafsson. On Resampling Algorithms for Particle Filters. _Nonlinear Statistical Signal Processing Workshop_ (2006). 

<sup>3</sup> N. Chopin, particles v0.1, (2021).[https://github.com/nchopin/particles](https://github.com/nchopin/particles).

## Load Experimental Data

In [2]:
## Assert File Names
inFile = '../04-Data/Bouc-Wen/inferenceInput'

infData = np.load(inFile + '.npz')

dt = infData['dt']                 # time step [sec]
time = infData['time']             # time history [sec]
inpAcc = infData['inpAcc']         # observations on input acceleration [m/sec^2]
states = infData['statesPNoise']   # states (for post-inference validation) [m,m/sec,m]
respAcc = infData['accPMNoise']    # observations on response acceleration [m/sec^2]
Q = infData['Qfactor']             # process noise contributions, independent std. dev. per state [m,m/sec,m]
R = infData['Rfactor']             # measurement noise contribution [m/sec^2]
m = infData['m']                   # mass [kg]
ics = infData['ics']               # true initial conditions of the system [m, m/sec, m]
par = infData['par']               # true parameters of the system [xi (-), wn (rad/sec), beta [1/m^2], n [-], gamma [1/m^2]] 

### Lay Out Problem Dimensionality ###
nInf = 8                     # Number of inferred variables [-]
nState = states.shape[0]     # Number of states [-]
nPar = nInf - nState         # Number of parameters [-]
samps = len(time)            # Number of system measurements [-]

### Define BW Model Class and Generate an Instance of the Class
Here's where I figure out how to turn the general example in this tutorial into a use case for the review paper. 

In [3]:
class BoucWen(ssm.StateSpaceModel):
    """
    The default parameters initialized below tell the model that we anticipate accepting
    parameters, but that they may be constants (particle filtering on the states) 
    or distributions (SMC^2 on the states and parameters). Those parameters which are
    given prior distributions and passed to the algorithm will be inferred, and the 
    remainder of these defaults will be treated as constants. 
    
    To accommodate not being able to modify the constants after I initialize the SMC^2 
    algorithm, I directly load the excitation into the initialization statement, so that
    all portions of the time history are accessible. This give a little more computational 
    overhead, but it should not be excessive, as we are putting this call in the class 
    initialization.
    """
    infData = np.load('../04-Data/Bouc-Wen/inferenceInput.npz')
    
    default_params = {'exc': infData['inpAcc'],
                      'dt': infData['dt'], 'Q':infData['Qfactor'], 'R':infData['Rfactor'],
                      'xi':np.log(0.05), 'wn':np.log(3.), 
                      'beta':np.log(2.), 'n':np.log(2.), 'gamma':np.log(1.)}
    # I've organized the default params dictionary such that the rows contain
    # (1) Input excitation
    # (2) Constant Parameters, including the time step, and noise terms
    # (3) Linear parameters to be inferred
    # (4) Nonlinear parameters to be inferred
    
    def PX0(self):
        """
        Defines the prior distribution on the states. 
        """
        mu0 = np.zeros(3)
        sig0 = np.square(np.array([0.05, 0.05, 0.05]))
        return dists.MvNormal(loc=mu0, scale=sig0, cov=np.eye(3))
    
    def PX(self, t, xp):
        """
        Defines the state transition function. 
        
        t = time index [-]
        xp = previous state estimate
        """
        ## Remove Transformation on the Parameters ##
        par = np.array([np.exp(self.xi), np.exp(self.wn), 
                        np.exp(self.beta), np.exp(self.n), np.exp(self.gamma)])

        ## Define State Transition ##
        x1dot = xp[0,0] + self.dt*xp[0,1]
        x2dot = xp[0,1] + self.dt*(-self.exc[t] - (2*par[0]*par[1])*xp[0,1] 
                            - np.square(par[1])*xp[0,2])
        x3dot = xp[0,2] + self.dt*(xp[0,1] - par[2]*np.absolute(xp[0,1])*
                    np.power(np.absolute(xp[0,2]), par[3]-1)*xp[0,2] 
                    - par[4]*xp[0,1]*np.power(np.absolute(xp[0,2]), par[3]))
        
        ## Format Results ##
        muT = np.stack((x1dot, x2dot, x3dot), axis=0)
        sigT = np.square(self.Q)
        return dists.MvNormal(loc=muT, scale=sigT, cov=np.eye(3))
    
    def PY(self, t, xp, x):
        """
        Defines the observation function.
        
        t = time index [-]
        xp = previous state estimate
        x = current state estimate
        """
        ## Remove Transformation on the Parameters ##
        par = np.array([np.exp(self.xi), np.exp(self.wn)])
         
        ## Define Obervation Function ##
        muR = -(2*par[0]*par[1])*x[:,1] - np.square(par[1])*x[:,2]
        sigR = np.square(self.R)
        return dists.Normal(loc=muR, scale=sigR)

In [4]:
def fx(x, dt, exc=None):
    """
    State transition model for a SDOF oscillator with a Bouc-Wen switch 
    state, given that alpha = 0, and therefore the Bouc-Wen component is 
    switched off.
      
    x = 1x8 vector of states (disp [m], vel [m/sec], Bouc-Wen disp [m]) 
                and parameters to be inferred (log(xi),log(wn), log(beta), 
                log(n), log(gamma)). 
    dt = sampling rate [sec]
    exc = input excitation at current time step [m/sec^2]
    """
    if exc is None:
        exc = np.zeros(x[1].shape)
      
    par = np.exp(x[3:]) 
            
    x1dot = x[0] + dt*x[1]
    x2dot = x[1] + dt*(-exc - (2*par[0]*par[1])*x[1] - np.square(par[1])*x[2])
    x3dot = x[2] + dt*(x[1] - par[2]*np.absolute(x[1])*
                np.power(np.absolute(x[2]), par[3]-1)*x[2] 
                - par[4]*x[1]*np.power(np.absolute(x[2]), par[3]))

    return np.concatenate((np.stack((x1dot, x2dot, x3dot), axis=0)
                           , x[3:]), axis=0)

## Run SMC$^2$

In [6]:
### PF Specific Parameters ###
# Changing the number of particles here will generate 
# separate save files. These can be combined into a full
# example case on all 50 prior distributions using the 
# 03.1-Compile_PF_Runs code. 
N_theta = 10          # Number of particles on the parameters
acceptRate = 0.1      # acceptance rate of the PMMH kernel
N_x = 50              # Number of particles on the states
resamp = 'systematic' # resampling type
ESS_thresh = 0.5      # effective sample size threshold

seed = 42568                            # Random seed to initialize stochastic sampling
checkStep = len(inpAcc)/5               # Meter the progress through the data              
outFile = ('../04-Data/Linear/outputPF_Nx' + 
           str(N_x) + '_Ntheta' + str(N_theta)) # output file

### Load Prior Distributions on the Parameters ###
parPriors = np.loadtxt('../04-Data/parameter_priors.txt')
parMeans = parPriors[:,::2]
parStds = parPriors[:,1::2]

### Generate Storage Over Inferred States/Parameters ###
muHist = np.zeros((parPriors.shape[0],nInf, samps))
    # mean of the inferred parameters for each inference trial
    # over the observation period. This is what the PF directly
    # outputs
stdHist = np.zeros((parPriors.shape[0],nInf, samps))
stdHist[:,:3,0] = 0.05*np.ones((parPriors.shape[0], nState))
    # standard deviation of the inferred parameters for each 
    # inference trial over the observation period. This is 
    # what the PF directly outputs
meanHist = np.zeros((parPriors.shape[0],nInf, samps))
    # mean of the inferred states and the underlying parameters
    # for each inference trial over the observation period. This
    # measure transforms the parameters to a lognormal distribution
    # such that the statistics on the true parameter values can be
    # extracted. 
modeHist = np.zeros((parPriors.shape[0],nInf, samps))
    # mode of the inferred states and the underlying parameters
    # for each inference trial over the observation period. This
    # measure transforms the parameters to a lognormal distribution
    # such that the statistics on the true parameter values can be
    # extracted. 
modStates = np.zeros((parPriors.shape[0],nInf, samps))
    # Response history of the inferred system given the input
    # excitation. Essentially, we're remodeling the behavior of 
    # the system given our selections on point estimates of the 
    # parameters from the posterior. 
runTime = np.zeros((parPriors.shape[0]))
    # Computational time for each inference trial. 
Nx = np.zeros((parPriors.shape[0],samps))
    # Tracks the increase in number of particles on the parameters
    # over the inference history. 

In [ ]:
## For Each Inference Trial... ##
for j in range(parPriors.shape[0]):
    ### Set a Unique Random Seed ###
    np.random.seed(seed+j*7)
    
    ### Structure the Prior Distributions on the Parameters ###
    prior_dict = {'xi': dists.Normal(loc=parMeans[j,0], scale=parStds[j,0]), 
                  'wn': dists.Normal(loc=parMeans[j,1], scale=parStds[j,1]), 
                  'beta': dists.Normal(loc=parMeans[j,2], scale=parStds[j,2]), 
                  'n': dists.Normal(loc=parMeans[j,3], scale=parStds[j,3]), 
                  'gamma': dists.Normal(loc=parMeans[j,4], scale=parStds[j,4])}
    my_prior = dists.StructDist(prior_dict)
    
    ### Initialize Filter Layers ###
    bwMod_smc2 = ssp.SMC2(ssm_cls=BoucWen, data=respAcc, prior=my_prior,init_Nx=N_theta, 
                       ar_to_increase_Nx=acceptRate)
                        # By default, we are asserting that the importance distributions
                        # be formed according to the 'Bootstrap' methodology, i.e. that
                        # the transition density forms the importance distribution
    bwSMC2 = particles.SMC(fk=bwMod_smc2, N=N_x, resampling = resamp,
                        ESSrmin = ESS_thresh, moments=True)

    ### Run Filter ###
    print('Iteration %d:'%(j))
    
    ## Time Each Inference Trial ##
    t0 = timeit.time()
    tf = t0
    
    for i in range(len(inpAcc)):
        ##  Check Filter Convergence ##
        # For this problem, run time for a single case is < 8 hours 
        if ((tf-t0)/3600 > 8):
            break
            
        ## Step the SMC^2 Filter Forward ##
        next(bwSMC2)
        
        ## Record Marginal Posterior on the States given Y_t, t<T ##
        muHist[j,:3,i] = np.average(np.array([np.average(pf.X, weights=pf.W, axis=0) for pf in bwSMC2.X.pfs]), 
                                 weights=bwSMC2.W, axis=0) 
        stdHist[j,:3,i] = np.sqrt(np.average(np.array([np.average(np.square(pf.X), 
                                                        weights=pf.W, axis=0) for pf in bwSMC2.X.pfs]), 
                                 weights=bwSMC2.W, axis=0) - np.square(muHist[j,:3,i])) 
        meanHist[j,:3,i] = muHist[j,:3,i]
        modeHist[j,:3,i] = muHist[j,:3,i]
        
        ## Report Progress through the Time History ##
        if (i+1)%checkStep == 0:
            now = datetime.datetime.now()
            print('\tI am {} seconds in at time {}'.format((i+1)*dt, now))
            print('\t\tNx = %d'%(bwSMC2.X.Nxs[i]))

        tf = timeit.time()
   
    ## If SMC^2 Fails to Converge ##
    if muHist[j,3,-1] == 0:
        print('\tTrial Exceeded Alloted Computation Time.')
        ## Store Results ##
        for i in range(1,samps):
            muHist[j,:,i] = np.array([0,0,0, -0.1054, 2.3, 3.4, 1.9, 3.4])
            stdHist[j,:,i] = 0.01*np.ones(8)
            meanHist[j,:,i] = np.concatenate((muHist[j,:nState,i], 
                                    np.exp(muHist[j,nState:,i] + np.square(stdHist[j,nState:,i])/2)))
            modeHist[j,:,i] = np.concatenate((muHist[j,:nState,i], np.exp(muHist[j,nState:,i] 
                                    - np.square(stdHist[j,nState:,i]))))
      
    ## If SMC^2 Converges ##
    else:
        ## Adapt Parameter Results to Array Format ## 
        pfMeans = np.array([m['mean'] for m in bwSMC2.summaries.moments])
        pfVars = np.array([m['var'] for m in bwSMC2.summaries.moments])

        for i in range(len(pfMeans)):
            muHist[j,3:,i] = np.array([pfMeans[i][4], pfMeans[i][3], pfMeans[i][0], pfMeans[i][2], pfMeans[i][1]])
            stdHist[j,3:,i] = np.sqrt(np.array([pfVars[i][4], pfVars[i][3], pfVars[i][0], pfVars[i][2], pfVars[i][1]]))
            meanHist[j,3:,i] = np.exp(muHist[j,3:,i] + np.square(stdHist[j,3:,i])/2)
            modeHist[j,3:,i] = np.exp(muHist[j,3:,i] - np.square(stdHist[j,3:,i]))
    
    ### Rerun Model with Point Estimates (Mode) of Inferred Parameters ###
    modStates[j,:,0] = np.concatenate((np.zeros((3,)), np.log(modeHist[j,3:,-1])))
    for i in range(1,len(time)):
        modStates[j,:,i] = fx(modStates[j,:,i-1], dt, exc=inpAcc[i-1])   

    ## Store Progression of Theta-Particles and RunTime ##
    Nx[j,:] = sdofSMC2.X.Nxs
    runTime[j] = (tf-t0)/60
    
    ## Print Results Summary and Save ##
    print('\tComputation Time = %d minutes and %d seconds' %(np.floor((tf-t0)/60), 
                                                        np.floor((tf-t0) - 60*np.floor((tf-t0)/60))) )
    print('\tMode of Final Parameter Distributions: \n\t\txi = %.4f,\n\t\twn = %.4f,\n\t\tbeta = %.4f,\n\t\tn = %.4f,\n\t\tgamma = %.4f\n'
          %(modeHist[j,3,-1],modeHist[j,4,-1],modeHist[j,5,-1],modeHist[j,6,-1],modeHist[j,7,-1]))

    ## Save Outputs After Each Inference Trial ##
    np.savez(outFile, muHist = muHist,stdHist=stdHist, meanHist=meanHist, 
             modeHist=modeHist, modStates = modStates, Nxs = Nx, runTime = runTime)

## Predictive Capacity of the Inferred Models
The goal of this section is to develop a prediction of the response behavior of the system to a secondary event, given the models which have been inferred from the primary excitation. 

### Load Inference Data
This becomes an optional start point in the code. If the data for the PF has already been generated, it can simply be loaded in for the predictive analysis instead of rerunning the previous block of code. 

In [8]:
outFile = '../04-Data/Bouc-Wen/outputPF'  # This output file is generated by combining all output
                                        # cases on N_theta and N_x. 
outData = np.load(outFile + '.npz')

muHist = outData['muHist']         # inference history of untransformed state/par means
stdHist = outData['stdHist']       # inference history of state/par standard deviations
meanHist = outData['meanHist']     # inference history of transformed state/par means
modeHist = outData['modeHist']     # inference history of transformed state/par modes
modStates = outData['modStates']  # states that have been remodeled based on the final modes of the parameters

### Load Secondary Input Excitation

In [9]:
predInFile = '../04-Data/Bouc-Wen/predInp_BLWN'
predOutFile = '../04-Data/Bouc-Wen/predOutPF'

infData = np.load(predInFile + '.npz')

dt = infData['dt']                            # time step [sec]
time = infData['time']                        # time history [sec]
predBase = infData['predInp']                 # observations on input acceleration [m/sec^2]
predStatesTrue = infData['predStatesPNoise']  # states (for post-prediction validation) [m,m/sec]
predRespTrue = infData['predAccPMNoise']      # observations on response acceleration [m/sec^2]
Q = infData['Qfactor']                        # process noise contributions, independent std. dev. per state [m,m/sec]
R = infData['Rfactor']                        # measurement noise contribution [m/sec^2]
m = infData['m']                              # mass [kg]
ics = infData['ics']                          # true initial conditions of the system [m, m/sec]
par = infData['par']                          # true parameters of the system [xi (-), wn (rad/sec)] 

### Generate Predictive Distribution on the States over Secondary Input

In [11]:
### Set Constants for Predictive Sampling ###
nPriors = muHist.shape[0]    # Number of inference trials [-]
nSamps = 500                 # Number of samples on the inference posterior [-]
seeds = [8192,3245]          # seeds for random number generator

### Generate Storage Over Predicted States ###
totalSamps = np.zeros((nSamps*nPriors, nState, len(time)))
    # Predicted states based on simulations results for all posterior
    # samples from all inference trials
meanPred = np.zeros((nPriors, nState, len(time)))
    # Mean of the predicted states for each inference trial.
stdPred = np.zeros((nPriors,nState, len(time)))
    # Standard deviation of the predicted states for each inference trial. 

### Run Predictive Trials on All Candidate Models ###
print('Predictive Distribution from Inferred Results')
for j in range(nPriors):
    print('Case %d'%(j))
    ## Random Samples on the States and Parameters, based on Inferred Posterior ##
    np.random.seed(seeds[0]+j)
    rSamp = np.random.multivariate_normal(np.zeros(nInf), np.eye(nInf), nSamps)
    predSamps = muHist[j,:,-1] + stdHist[j,:,-1]*rSamp

    ## Random Samples on the Transition Noise ##
    np.random.seed(seeds[1]+j)
    noise = Q.reshape(-1,1)*np.random.multivariate_normal(np.zeros(nState), np.eye(nState), 
                                                          (nSamps, len(time))).transpose((0, 2, 1))

    ## Prepare Response Storage ##
    predStates = np.zeros((nSamps, nState,len(time)))
    predStates[:,:,0] = predSamps[:,:nState]

    for i in range(nSamps):
        for tt in range(1,len(time)):
            predStates[i,:,tt] = fx(np.concatenate((predStates[i,:, tt-1], predSamps[i,nState:])), 
                                                 dt, exc = predBase[tt-1])[:nState] + noise[i,:,tt-1]
    
    ## Store Results from Predictive Sample Runs ##
    meanPred[j,:,:] = np.mean(predStates, axis = 0)
    stdPred[j,:,:] = np.sqrt(np.mean(np.square(predStates), axis=0) - np.square(meanPred[j,:,:])) 
    totalSamps[j*nSamps:(j+1)*nSamps,:,:] = predStates
    
### Remove Unstable Results from the Overall Assessment ###
# Candidate models can become unstable during inference (due to 
# computational issues such as singularities in the covariance 
# matrices) or manifest instability during predictive modeling
# due to combinations of the selected parameters which result in
# model divergence. Here we extract these cases so that they don't 
# interfere with the statistics of the main results. 
stabilityInd = np.ones(nPriors)
totalStabilityInd = np.ones(nPriors*nSamps)

print('\nIndices of Unstable Predictive Distributions:')
for i in range(nPriors):
    if (np.isnan(meanPred[i,0,-1])) or (np.absolute(meanPred[i,0,-1])>100) or (muHist[i,0,-1] == 0):
        stabilityInd[i] = 0 
        totalStabilityInd[i*nSamps:(i+1)*nSamps] = np.zeros(nSamps)
        print(i)

stableMeans = meanPred[stabilityInd != 0,:,:]
stableStds = stdPred[stabilityInd != 0,:,:]
stableSamps = totalSamps[totalStabilityInd != 0,:,:]

### Statistics on all Stable Cases ###
meanAll = np.mean(stableSamps, axis = 0)
stdAll = np.sqrt(np.mean(np.square(stableSamps), axis=0) - np.square(meanAll)) 

### Save Output ###
np.savez(predOutFile, meanPred = meanPred,stdPred=stdPred, 
         stableMeans=stableMeans, stableStds=stableStds, meanAll=meanAll, stdAll=stdAll)

Predictive Distribution from Inferred Results
Case 0
Case 1
Case 2
Case 3
Case 4
Case 5
Case 6
Case 7
Case 8
Case 9
Case 10
Case 11
Case 12
Case 13
Case 14
Case 15
Case 16
Case 17
Case 18
Case 19
Case 20
Case 21
Case 22
Case 23
Case 24
Case 25
Case 26
Case 27
Case 28
Case 29
Case 30
Case 31
Case 32
Case 33
Case 34
Case 35
Case 36
Case 37
Case 38
Case 39
Case 40
Case 41
Case 42
Case 43
Case 44
Case 45
Case 46
Case 47
Case 48
Case 49

Indices of Unstable Predictive Distributions:
4
20
28
30
38
46
48
